# Complete Solution

This notebook that analysis the Crown.

---------------

##### Imports

In [ ]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [ ]:
# Import modules.
import numpy as np
import open3d as o3d

from tree import analyse_tree
import utils.o3d_utils as o3d_utils

##### Load data

In [ ]:
# Load point cloud data
source = 'cyclo'

if source == 'ahn':
    pcd = o3d_utils.read_las('../datasets/single_selection/single_121913_487434_AHN.las')
elif source == 'cyclo':
    pcd = o3d_utils.read_las('../datasets/single_selection/single_121913_487434_Cyclo.las')
else:
    pcd = o3d_utils.read_las('../datasets/single_selection/single_121913_487434_Sonarski.las')


##### Experimental

In [ ]:
adTree_exe = '../../AdTree-single/build/bin/AdTree.app/Contents/MacOS/AdTree'
result = analyse_tree(pcd, adTree_exe, filter_leaves='curvature', crown_model_method='Convex_Hull')

In [ ]:
result

In [ ]:
o3d.visualization.draw_geometries([result['crown_mesh'],result['stem_mesh']])